# Part 2 - Multiple Scenario Analysis

<div class="alert alert-block alert-info">Before you run this notebook, you must run Part 1 - Network Design`.</div>
</div>

## Set up

#### Insert a project token

When you import this project from the Watson Studio Gallery, a token should be automatically generated and inserted at the top of this notebook as a code cell such as the one below:

```python
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='YOUR_PROJECT_ID', project_access_token='YOUR_PROJECT_TOKEN')
pc = project.project_context
```

If you do not see the cell above, follow these steps to enable the notebook to access the dataset from the project's resources:

* Click on `More -> Insert project token` in the top-right menu section

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)

* This should insert a cell at the top of this notebook similar to the example given above.

  > If an error is displayed indicating that no project token is defined, follow [these instructions](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/token.html?audience=wdp&context=data).

* Run the newly inserted cell before proceeding with the notebook execution below

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
!pip install decision-optimization-client

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for decision-optimization-client: filename=decision_optimization_client-1.0.0-py3-none-any.whl size=32039 sha256=1a2beb83760caf39dacb7e5cfb8d490cf5397553b27277f63ae250e50734c264
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/d9/6c/2b/df5659588cc193bb5ae95c0ed5c00b021335ec25c523a4642d
Successfully built decision-optimization-client


## Import the libraries

In [1]:
# This notebook requires an APIKEY to be able to use WML to solve the different scenarios from the Decision Optimization experiment.
# To access your API key, go to cloud.ibm.com, then click Manage > Access(IAM)

apikey=""

In [6]:
from decision_optimization_client import *

# In order to use the solve() function, you must provide an API key when creating the client
pc = project.project_context
client = Client(pc=pc, apikey=apikey)
decision = client.get_experiment(name="NetworkDesign")
scenario = decision.get_scenario(name="Scenario 1")
df_parameters = scenario.get_table_data('PARAMETERS')
print (df_parameters)

            name  value
0   MAX_PRODUCTS      5
1  MAX_CUSTOMERS    300
2   MIN_OPEN_DCS      1
3   MAX_OPEN_DCS      3


## Analyzing multiple scenarios

In [7]:
min = 1
max = 10

for i in range(min, max+1):
    print()
    sc_name = "Fixed %02d" % (i)
    print(sc_name)
    copy = decision.get_scenario(name=sc_name)
    if (copy != None):
        print(" Deleting old...")
        decision.delete_container(copy)
    print(" Copying from original scenario...")    
    copy = scenario.copy(sc_name)
    print(" Modifying parameters...")
    df_parameters = copy.get_table_data('PARAMETERS')
    df_parameters.value[df_parameters.name=='MIN_OPEN_DCS'] = i
    df_parameters.value[df_parameters.name=='MAX_OPEN_DCS'] = i
    print (df_parameters)
    copy.add_table_data("PARAMETERS", df_parameters, category='input')
    print(" Solving...")
    copy.solve(log_output=True)
    
print("Done!")


Fixed 01
 Copying from original scenario...
 Modifying parameters...
            name  value
0   MAX_PRODUCTS      5
1  MAX_CUSTOMERS    300
2   MIN_OPEN_DCS      1
3   MAX_OPEN_DCS      1
 Solving...

Fixed 02
 Copying from original scenario...
 Modifying parameters...
            name  value
0   MAX_PRODUCTS      5
1  MAX_CUSTOMERS    300
2   MIN_OPEN_DCS      2
3   MAX_OPEN_DCS      2
 Solving...

Fixed 03
 Copying from original scenario...
 Modifying parameters...
            name  value
0   MAX_PRODUCTS      5
1  MAX_CUSTOMERS    300
2   MIN_OPEN_DCS      3
3   MAX_OPEN_DCS      3
 Solving...

Fixed 04
 Copying from original scenario...
 Modifying parameters...
            name  value
0   MAX_PRODUCTS      5
1  MAX_CUSTOMERS    300
2   MIN_OPEN_DCS      4
3   MAX_OPEN_DCS      4
 Solving...

Fixed 05
 Copying from original scenario...
 Modifying parameters...
            name  value
0   MAX_PRODUCTS      5
1  MAX_CUSTOMERS    300
2   MIN_OPEN_DCS      5
3   MAX_OPEN_DCS      5
 S

In [8]:
import pandas as pd

min = 1
max = 10

df_all_kpis = None
print(" Aggregating solution kpis in a unique table...")
for i in range(min, max+1):
    sc_name = "Fixed %02d" % (i)
    print(sc_name)
    copy = decision.get_scenario(name=sc_name)
    df_kpis = copy.get_table_data('kpis').copy()
    df_kpis = df_kpis.rename(columns={"Value": sc_name})
    df_kpis = df_kpis.set_index("Name")
    if i==min:
        df_all_kpis = df_kpis.copy()
    else:
        df_all_kpis = df_all_kpis.join(df_kpis)
    
df_all_kpis = df_all_kpis.reset_index()
scenario.add_table_data("all_kpis", df_all_kpis, category='output')
print(df_all_kpis)



 Aggregating solution kpis in a unique table...
Fixed 01
Fixed 02
Fixed 03
Fixed 04
Fixed 05
Fixed 06
Fixed 07
Fixed 08
Fixed 09
Fixed 10
                             Name     Fixed 01     Fixed 02     Fixed 03  \
0               variablePlantCost  15822450.00  16738050.00  16596500.00   
1       inboundTransportationCost   1928864.40   6762800.76   5107575.00   
2      outboundTransportationCost  33489241.44  12480266.58  10747613.62   
3     fixedDistributionCenterCost    500000.00   1150000.00   1650000.00   
4  variableDistributionCenterCost   2677000.00   2677000.00   2677000.00   
5       nbOpenDistributionCenters         1.00         2.00         3.00   

      Fixed 04     Fixed 05     Fixed 06     Fixed 07     Fixed 08  \
0  16598800.00  16603800.00  16603800.00  16598800.00  16418800.00   
1   5961815.04   6765971.68   7105138.72   7692070.13   8225918.19   
2   9170047.89   7741823.99   6627746.46   5619659.53   4922107.87   
3   2150000.00   2600000.00   3250000.00   370000

## Aggregating solutions

In [9]:
import pandas as pd

min = 1
max = 10

df_all_kpis_2 = pd.DataFrame()
print(" Aggregating solution kpis in a unique table...")
for i in range(min, max+1):
    sc_name = "Fixed %02d" % (i)
    print(sc_name)
    copy = decision.get_scenario(name=sc_name)
    df_kpis = copy.get_table_data('kpis').copy()
    df_kpis['Scenario'] = sc_name
    df_kpis = df_kpis.rename(columns={"Value": sc_name})
    df_kpis = df_kpis.pivot(index="Scenario", columns="Name", values=sc_name)
    #print (df_kpis)
    df_all_kpis_2 = df_all_kpis_2.append(df_kpis)
    
df_all_kpis_2 = df_all_kpis_2.reset_index()
scenario.add_table_data("all_kpis_2", df_all_kpis_2, category='output')
print(df_all_kpis_2)

 Aggregating solution kpis in a unique table...
Fixed 01
Fixed 02
Fixed 03
Fixed 04
Fixed 05
Fixed 06
Fixed 07
Fixed 08
Fixed 09
Fixed 10
Name  Scenario  fixedDistributionCenterCost  inboundTransportationCost  \
0     Fixed 01                     500000.0                 1928864.40   
1     Fixed 02                    1150000.0                 6762800.76   
2     Fixed 03                    1650000.0                 5107575.00   
3     Fixed 04                    2150000.0                 5961815.04   
4     Fixed 05                    2600000.0                 6765971.68   
5     Fixed 06                    3250000.0                 7105138.72   
6     Fixed 07                    3700000.0                 7692070.13   
7     Fixed 08                    4200000.0                 8225918.19   
8     Fixed 09                    4700000.0                 8577446.40   
9     Fixed 10                    5200000.0                 8876355.68   

Name  nbOpenDistributionCenters  outboundTransp

## Summary

Congratulations! You have learned how to:

1. Load and prepare network design data
2. Create a decision optimization model
3. Configure the KPIs, constraints and objectives
4. Solve the optimization problem
5. Create and analyze multiple scenarios
6. Load the solutions into a Decision Optimization experiment

To see the results, go to the project home and click on `NetworkDesign` under Decision Optimization experiment.

# <hr>
Copyright © 2021 IBM. This notebook and its source code are released under the terms of the MIT License.